In [ ]:
# Cell 1: Setup and imports
import sys
import os
from typing import Dict, List, Any
from datetime import datetime

# Add the parent directory to sys.path if needed
sys.path.append(os.path.abspath(".."))

# Import the workflow functions
from workflows.content_generation.ugc_script_gen import generate_ugc_scripts

In [ ]:
# --- RUN ---
content_strategy = {'brand_colors': '#2c3e50, #3498db, #f8f9fa',
 'brand_tone': 'Professional',
 'content_pillars': ['quality', 'benefits', 'sustainability'],
 'content_types': ['ad_copy',
                   'social_captions',
                   'static_images',
                   'ugc_scripts',
                   'email_creative'],
 'key_messages': ['Saves you time and effort', 'Premium quality you can trust'],
 'product_description': 'Premium organic skincare serum with natural '
                        'ingredients that helps improve skin texture and '
                        'reduce signs of aging. Perfect for busy professionals '
                        'who want quality skincare.',
 'target_audience': 'Busy professionals aged 25-45 who value quality and '
                    'natural products'}

generated_text = generate_ugc_scripts(content_strategy, brand_tone=content_strategy['brand_tone'], target_audience=content_strategy['target_audience'] )
for i in generated_text:
    print(i)

In [ ]:
from workflows.content_generation.ugc_script_gen import generate_single_ugc_script

type = "testimonial"
duration = "30 seconds"
purpose=  "Build trust and social proof"

generated_stext = generate_single_ugc_script(content_strategy, brand_tone=content_strategy['brand_tone'], target_audience=content_strategy['target_audience'], script_type=type, duration=duration, purpose=purpose )
for i, j in generated_stext.items():
    print(i, '=', j)

In [ ]:
from workflows.content_generation.ugc_script_gen import create_ugc_script_prompt

generated_prom = create_ugc_script_prompt(content_strategy, brand_tone=content_strategy['brand_tone'], target_audience=content_strategy['target_audience'], script_type=type, duration=duration, purpose=purpose )
print(generated_prom)

In [ ]:
from tools.llm_manager import get_llm_response

# Get script from LLM
script_response = get_llm_response(
    prompt=generated_prom,
    system_message="You are an expert UGC content creator who specializes in authentic, engaging video scripts that feel natural and convert viewers into customers."
)

print(script_response)

In [ ]:
from workflows.content_generation.ugc_script_gen import parse_ugc_script_response
final_output = parse_ugc_script_response(script_response, script_type=type, duration=duration, purpose=purpose)

In [ ]:

for i,j in final_output.items():
    print(i, "=" ,j)
print(final_output)

In [ ]:
from workflows.content_generation.ugc_script_gen import generate_script_title
title = generate_script_title(type, script_response)
print(title)

In [ ]:
def extract_visual_directions(script_text: str) -> List[str]:
    """Extract visual directions and camera instructions"""
    
    visual_directions = []
    lines = script_text.split('\n')
    
    # Keywords that indicate visual directions
    visual_keywords = [
        'show', 'camera', 'close-up', 'wide shot', 'zoom', 'cut to',
        'display', 'reveal', 'focus on', 'angle', 'shot', 'frame'
    ]
    
    for line in lines:
        line_clean = line.strip().lower()
        if any(keyword in line_clean for keyword in visual_keywords):
            visual_directions.append(line.strip())
    
    # Add default visual directions if none found
    if not visual_directions:
        visual_directions = [
            "Start with medium shot of person",
            "Show product prominently in frame",
            "Include close-ups of key features",
            "End with wide shot showing results"
        ]
    
    return visual_directions

visual = extract_visual_directions(script_response)

In [ ]:
print(visual)

In [ ]:
# Test case 1: Script with visual directions
script1 = """
The presenter starts with a close-up of the product.
Cut to a wide shot of the room.
Show the user interacting with the interface.
"""

visual1 = extract_visual_directions(script1)
print("Test 1 Visual Directions:", visual1)

In [ ]:
# Test case 2: Script with no visual directions
script2 = """
Today we talk about the benefits of the product.
It is very useful and cost-effective.
"""

visual2 = extract_visual_directions(script2)
print("Test 2 Visual Directions:", visual2)

In [ ]:
def generate_script_title(script_type: str, script_text: str) -> str:
    """Generate a catchy title for the UGC script"""
    
    # Look for title in the script text
    lines = script_text.split('\n')
    for line in lines[:5]:
        if 'title:' in line.lower():
            return line.split(':', 1)[1].strip()
    
    # Generate based on script type
    titles = {
        "testimonial": "My Honest Review",
        "product_demo": "Let Me Show You How This Works",
        "unboxing": "Unboxing My New Favorite Thing",
        "before_after": "The Results Speak for Themselves"
    }
    
    return titles.get(script_type, f"{script_type.title()} Script")

# Test 1: Title explicitly present in the first 5 lines
script1 = """
Title: Why I Love This Product
This product has changed my life because...
"""

print(generate_script_title("testimonial", script1))
# Expected: "Why I Love This Product"

# Test 2: No title, known script type
script2 = """
This product demo will show you the features...
"""

print(generate_script_title("product_demo", script2))
# Expected: "Let Me Show You How This Works"

# Test 3: No title, unknown script type
script3 = """
Just some text without a title.
"""

print(generate_script_title("random_type", script3))
# Expected: "Random_Type Script"


In [ ]:
from typing import List, Dict

def extract_time_value(text: str) -> str:
    """Extract time value from text"""
    import re
    
    # Look for time patterns
    time_patterns = [r'\d+:\d+', r'\d+\s*sec', r'\d+-\d+', r'\d+s']
    
    for pattern in time_patterns:
        match = re.search(pattern, text)
        if match:
            return match.group()
    
    return "TBD"

def extract_timing_cues(script_text: str) -> List[Dict[str, str]]:
    """Extract timing and pacing cues"""
    
    timing_cues = []
    lines = script_text.split('\n')
    
    # Look for timing indicators
    timing_indicators = ['0:', '1:', '2:', 'seconds', 'sec', 'start', 'end', 'beginning']
    
    for line in lines:
        line_lower = line.lower()
        if any(indicator in line_lower for indicator in timing_indicators):
            timing_cues.append({
                "timing": extract_time_value(line),
                "action": line.strip(),
                "importance": "critical"
            })
    
    # Add default timing structure
    if not timing_cues:
        timing_cues = [
            {"timing": "0-3 sec", "action": "Hook - grab attention", "importance": "critical"},
            {"timing": "3-15 sec", "action": "Main content - demonstrate/explain", "importance": "high"},
            {"timing": "15+ sec", "action": "Call to action - encourage engagement", "importance": "critical"}
        ]
    
    return timing_cues

# Test inputs
script1 = """
0: Start with a hook to grab attention.
5 seconds: Show the product features.
End: Call to action with website link.
"""

script2 = """
Welcome to the product demo.
This product is great for everyone.
Get yours today!
"""

print("Test 1 timing cues:")
for cue in extract_timing_cues(script1):
    print(cue)

print("\nTest 2 timing cues (should be default):")
for cue in extract_timing_cues(script2):
    print(cue)


In [ ]:
from typing import List

def extract_props_needed(script_text: str) -> List[str]:
    """Extract props and materials needed for the video"""
    
    props = []
    
    # Look for prop-related keywords
    prop_keywords = ['prop', 'need', 'use', 'grab', 'hold', 'wear', 'setup']
    
    lines = script_text.split('\n')
    for line in lines:
        line_lower = line.lower()
        if any(keyword in line_lower for keyword in prop_keywords) and 'product' in line_lower:
            props.append(line.strip())
    
    # Add common UGC props
    common_props = [
        "The actual product",
        "Good lighting setup",
        "Clean background",
        "Phone or camera for recording"
    ]
    
    props.extend(common_props)
    
    return list(set(props))  # Remove duplicates


# Test case 1: Script with explicit prop mentions
script1 = """
First, grab the product and place it on the table.
You need to wear the product to demonstrate its use.
Hold the product in good lighting.
"""

print("Test 1 props:")
print(extract_props_needed(script1))

# Test case 2: Script with no explicit prop mentions
script2 = """
Welcome to the unboxing video.
We will open the package and show what's inside.
"""

print("\nTest 2 props (should only have common defaults):")
print(extract_props_needed(script2))


In [ ]:
from typing import List

def extract_setup_requirements(script_text: str) -> List[str]:
    """Extract setup and preparation requirements"""
    
    setup_requirements = []
    
    # Look for setup-related content
    setup_keywords = ['setup', 'prepare', 'before', 'location', 'lighting', 'background']
    
    lines = script_text.split('\n')
    for line in lines:
        line_lower = line.lower()
        if any(keyword in line_lower for keyword in setup_keywords):
            setup_requirements.append(line.strip())
    
    # Add standard UGC setup requirements
    standard_setup = [
        "Ensure good natural lighting or ring light",
        "Choose clean, uncluttered background",
        "Test audio quality and minimize background noise",
        "Have product easily accessible and ready to use",
        "Charge phone/camera and clear storage space"
    ]
    
    setup_requirements.extend(standard_setup)
    
    return setup_requirements


# Test 1: Script with setup instructions
script1 = """
Before starting, make sure to set up your camera properly.
Check the lighting in your room.
Prepare a clean background area.
"""

print("Test 1 setup requirements:")
for item in extract_setup_requirements(script1):
    print("-", item)

# Test 2: Script without setup-specific keywords
script2 = """
Today we will talk about the product features.
Make sure to hold the product close to the camera.
"""

print("\nTest 2 setup requirements (should only have standard setup):")
for item in extract_setup_requirements(script2):
    print("-", item)


In [ ]:
from typing import List

def extract_editing_notes(script_text: str) -> List[str]:
    """Extract editing and post-production notes"""
    
    editing_notes = []
    
    # Look for editing-related instructions
    edit_keywords = ['edit', 'cut', 'transition', 'music', 'sound', 'caption', 'text']
    
    lines = script_text.split('\n')
    for line in lines:
        line_lower = line.lower()
        if any(keyword in line_lower for keyword in edit_keywords):
            editing_notes.append(line.strip())
    
    # Add standard UGC editing notes
    standard_edits = [
        "Keep cuts snappy and engaging",
        "Add captions for accessibility",
        "Include trending music if appropriate",
        "Maintain authentic, unpolished feel",
        "Add subtle text overlays for key points"
    ]
    
    editing_notes.extend(standard_edits)
    
    return editing_notes

# Test 1: Script with editing instructions
script1 = """
Remember to add smooth transitions between scenes.
Cut out any long pauses.
Include background music to set the mood.
Add captions for clarity.
"""

print("Test 1 editing notes:")
for note in extract_editing_notes(script1):
    print("-", note)

# Test 2: Script without explicit editing instructions
script2 = """
This is the product demo.
Show the product in use.
"""

print("\nTest 2 editing notes (should only have standard notes):")
for note in extract_editing_notes(script2):
    print("-", note)
